# Introduction to Linear Regression (Part 1)

In this tutorial, we familiarize ourselves with one of the most important problem formulations in machine learning, _regression_. To acquaint ourselves with the richness of this approach, we will specifically develop a simpler setting called _linear regression_ from three distinct perspectives - _optimization_, _linear algebra_, and a _probabilistic model_. Along the way, we'll find introduce ourselves to a variety of numerical and plotting functions.

## Conda Environment Setup

Here is a sample `environment.yml` file that will help setup our Conda environment.

```yml
name: nyu-curp-2021
channels:
  - conda-forge
dependencies:
  - python
  - pip
  - pip:
    - numpy
    - scipy
    - scikit-learn
    - jupyterlab
    - seaborn
```

Most notably, we rely on a few key packages (highlighted above):

- `numpy` and `scipy`: For various vectorized numerical routines.
- `jupyterlab`: For interactive experimentation.
- `scikit-learn`: For implementation of the most popular machine learning algorithms.
- `seaborn`: For easy plotting.

To install these packages in a _new_ environment, run

```shell
conda env create -f environment.yml
```

**NOTE**: If you have an existing environment with such a file, it is sufficient to ensure all the packages listed above are present. If not, add them to the existing file and run
  ```shell
  conda env update -f environment.yml
  ```

### Common Imports

In [ ]:
import numpy as np
import scipy as sp
import seaborn as sns
from sklearn.linear_model import LinearRegression

sns.set(font_scale=1.25, style='whitegrid', palette=sns.color_palette("Set1"))

## Regression Problem Setup

> The goal of regression is to predict one or more continuous target variables $y$ given a set of $d$-dimensional input variables (predictors) $\mathbf{x} \in \mathbb{R}^{d}$. In a typical setting, we will generally have $n$ training samples, which we will index by subscripts $[ \mathbf{x}_1, \mathbf{x}_2, \dots, \mathbf{x}_n ]$, with each having a corresponding target observed value, $[ y_1, y_2, \dots, y_n]$.

We will restrict ourselves to one-dimensional outputs for this tutorial, i.e. $y \in \mathbb{R}$.

**Where do we really use regression?** Regression is a potential candidate to formulate any problem that demands predicting continuous variables -- a sales forecast where $\mathbf{x}$ is time and $y$ is the dollar amount of sales, predicting bounding boxes for object detection in images where $\mathbf{x}$ is the image and $y$ is the coordinates of the bounding box, predicting the steering angle in a self-driving car based on the observed image where $\mathbf{x}$ is the image and $y$ is the steering angle. This formulation is very general, and the application space remains vast. The most important point, however, is to be aware of the underlying statistical assumptions.

We will specifically focus on _linear_ regression, and clarify what's linear about it.

## A Toy Problem

Let us generate $100$ points between $0.0$ and $10.0$, to represent our inputs. We will use [`np.linspace`](https://numpy.org/doc/stable/reference/generated/numpy.linspace.html) to do this. We will pick a slope as $m = 2.0$ and intercept $c = 5.0$ to generate points for the line.

$$
y = mx + c
$$

**TIP**: A good way to pull up documentation (when available) for any function from within Jupyter notebooks is executing `?<function>`, for instance `?np.linspace`.

In [ ]:
# ?np.linspace

In [ ]:
m = 2.0
c = 5.0

x = np.linspace(0.0, 10.0, 100)
y = m * x + c

x.shape, y.shape

We use [`seaborn`](https://seaborn.pydata.org) for an easy interface to plotting. Underneath, it still uses [`matplotlib`](https://matplotlib.org), and therefore remains just as flexible.

In [ ]:
g = sns.relplot(x=x, y=y, kind='line')
g.ax.set_xlabel('x')
g.ax.set_ylabel('y');

Gives these set of $\mathbf{x}$ and $y$, we would like to figure out the values for the slope $m$ and $c$. In machine learning, we typically label any such process of figuring out the parameters (potentially infinite) of the underlying function as "learning".

**NOTE**: Knowing the true values beforehand will allow us to assess our learning for now. In a real world setting, however, we will not have such a foresight. Even worse, in most non-trivial settings, we cannot even be sure whether the data comes from a line (or a linear model in general), but we defer this discussion for later.

## Solution Perspectives

We discuss three perspectives, to realize a flavor of the richness of even a simple linear regression formulation.

### Optimization

Let us posit that we want to learn a linear function $f_\theta$ which has parameters $\theta = \{\theta_1,\theta_2\}$, such that

$$
f_\theta(x) = \theta_1 \times x + \theta_2
$$

> The linearity is meant to convey _linearity in the parameters_, and not linearity in the inputs. The inputs can be transformed by any operation $\mathbf{x} \to \phi(\mathbf{x})$, as long as the operation is fixed a-priori.

Now, we need some quantifiable measure to see how far a given $f_\theta$ is from our data set containing $(x, y)$. A popular choice is the error defined by the $L_2$-norm. The squared error $L_2$-norm of a $d$-dimensional vector $\mathbf{v} = [v_1,\dots,v_d]$ (each $v_i$ is a scalar) is defined as

$$
\ell_2(\mathbf{v}) = \sum_{i=1}^d v_i^2
$$

We call our quantifiable measure, the "loss", as the difference between the value predicted by our function $f_\theta$ and $y$ from our data, at the given input value $x$. We want to minimize this over all our dataset, such that the expression for the _sum-of-squares_ loss is,

$$
\mathcal{L}(\theta, [\mathbf{x}_1,\dots,\mathbf{x}_n],[y_1,\dots,y_n]) = \sum_{i=1}^n \left( f_\theta(\mathbf{x}_i) - y_i \right)^2
$$

Now, one can now use their favorite solver to optimize this objective. In terms of the code, `scikit-learn` provides us with a simple set of interfaces and a rich set of implemented algorithms to solve such a task. With this, linear regression is just a handful of lines of code.

In [ ]:
model = LinearRegression()
model.fit(np.expand_dims(x, axis=-1), y)
model.coef_, model.intercept_

The [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) interface provides us with a way to recover the learned parameters. In this case, `model.coef_` gives us the coeffient $\theta_1$, and `model.intercept_` gives us the intercept $\theta_2$. We have exactly recovered our true parameters!

**NOTE**: The `.fit()` interface requires that the inputs be two-dimensional. Our original `x` variable was a one-dimensional array of size `100`. We use [`np.expand_dims`](https://numpy.org/doc/stable/reference/generated/numpy.expand_dims.html) to add an additional dimension, such that the input is of size $100 \times 1$.

### Linear Algebra

Often, across machine learning, thinking in terms of vectors and vector spaces is fruitful. We can first rewrite $f_\theta$ as a matrix product between an augment input vector $\mathbf{x}$ and the parameter vector $\theta$,

$$
f_\theta(x) = \overbrace{\begin{bmatrix}x & 1\end{bmatrix}}^{\mathbf{x}} \underbrace{\begin{bmatrix}\theta_1 \\ \theta_2\end{bmatrix}}_{\theta}.
$$

Notice that we have appended another column of ones, to account for the addition of the intercept. For our one-dimensional setting, the intercept is $\theta_2$. In general, we can rewrite all the inputs (predictors) into a $n \times (d+1)$ matrix $\mathbf{X}$, such that, each row contains a single input augment with $1$. Similarly, we rewrite all the outputs into a $n \times 1$ matrix such that each row contains a single output (here one-dimensional).

$$
\mathbf{X} = \begin{bmatrix} \cdots & \mathbf{x}_1 & \cdots & 1 \\ \cdots & \mathbf{x}_2 & \cdots & 1 \\  & \vdots &  & \vdots \\ \cdots & \mathbf{x}_n & \cdots & 1 \end{bmatrix}, \mathbf{y} = \begin{bmatrix}y_1 \\ y_2 \\ \vdots \\ y_n \end{bmatrix}.
$$

We can now express the function $f_\theta(x) = \theta_1 \times x + \theta_2$ as a matrix-vector multiplication, by overloading the function notation $f_\theta$ to take input as the full matrix $\mathbf{X}$, and the corresponding loss can be written as,

$$
f_\theta(\mathbf{X}) = \mathbf{X}\theta
$$

The loss can also be converted into our succinct matrix-vector notations as,

$$
\mathcal{L}(\theta,\mathbf{X},\mathbf{y}) = (\mathbf{X}\theta - \mathbf{y})^T(\mathbf{X}\theta - \mathbf{y}) = \theta^T\mathbf{X}^T\mathbf{X}\theta - \mathbf{y}^T\mathbf{X}\theta - \theta^T\mathbf{X}^T\mathbf{y} + \mathbf{y}^T\mathbf{y}
$$

**NOTE**: Notice how we represent the $L_2$-norm as an inner product between vectors. It is instructive to verify this is true.

We optimize this objective using the usual rules from calculus (although, applied to matrices form), by taking the first derivative w.r.t $\theta$

$$
\frac{d\mathcal{L}}{d\theta}(\theta,\mathbf{X},\mathbf{y}) = 2\mathbf{X}^T\mathbf{X}\theta - 2\mathbf{X}^T\mathbf{y} = \mathbf{0} \\
\implies \theta = (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y}
$$

We can now take our data, and directly implement this using `numpy` below. We do a few things here: first, we concatenate a column of ones to our inputs $x$. Then we expand the dimensions on $y$ so that we can do our usual matrix-vector multiplications.

In [ ]:
X = np.concatenate([
  np.expand_dims(x, axis=-1),
  np.ones((x.shape[0], 1))], axis=-1)
y = np.expand_dims(y, axis=-1)
X.shape, y.shape

`numpy` supports a rich set of operations, ranging from simple matrix multiplications, to more complex linear algebra operations like computing matrix decompositions and inverses. In this case, we use [`np.matmul`](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html) for matrix multiplications, and [`np.linalg.inv`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.inv.html) to compute the inverse.

In [ ]:
theta = np.matmul(np.linalg.inv(np.matmul(X.T, X)), np.matmul(X.T, y))
theta

We recover our true values exactly! This is not particularly surprising, given that we derived a closed-form expression from first principles. 

**NOTE**: It is important to be aware that solving linear equations containing inverses this way is not recommended with real-world applications. There are numerically more stable methods, that also scale better with the size of the matrix. Further, the inverse may not always be possible in the first place, and often need to be conditioned with a _regularizer_. These topics remains beyond the scope of our tutorial.

**What does this expression for $\theta$ really imply?** To see the implication of this result, we can view it from the lens of orthogonal projectors - the solution $\theta$ here is chosen such that $\mathbf{X}\theta$ forms an orthogonal projection of the output vector $\mathbf{y}$ onto the subspace spanned by the columns of $\mathbf{X}$. For the familiar eye, an orthogonal projector for an arbitrary basis is defined by $\mathbf{X}(\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T$.

### Probabilistic Modeling

In the real world, we often do not have the luxury of noise-free or exact measurements in the true sense. Often, our measurement device may have internal errors. For instance, the weighing scale at home is often riddled with measurement errors. 

Instead, in the probabilistic land, one can always argue that with only a finite number of measurements, all we have is a probability "distribution" over weights. The true weight can only be recovered in the _limit_ of infinite data, i.e. you must keep measuring your weight forever. Intuitively, the probability distribution over weights can be understood as assigning a non-negative _score_ to each plausible value of the weight measurement. A score of zero would imply implausibility, a weight that is surely not the true value.

A common first-choice (not necessarily the best in many cases) for continuous values is the Gaussian distribution. A Gaussian distribution over a random variable $x$, $\mathcal{N}(x; \mu,\sigma^2)$, is specified by a mean $\mu$ and a variance $\sigma^2$. In this case, the mean defines where the distribution is centered (not true always), and variance represents the spread of the values.

$$
p(x) = \mathcal{N}(x; \mu, \sigma^2) = \frac{1}{\sqrt{2\pi\sigma^2}}\exp\left\{-\frac{1}{2} \frac{(x - \mu)^2}{\sigma^2}\right\}
$$

We visualize a few values of the variance, for distributions centered around $0.0$.

In [ ]:
gx = np.linspace(-5.0, 5.0, 500)
g = sns.relplot(x=gx, y=sp.stats.norm(loc=0.0, scale=1.0).pdf(gx))
g.ax.clear()
for scale in [0.25, 0.5, 1.0, 2.0, 4.0]:
    sns.lineplot(x=gx, y=sp.stats.norm(loc=0.0, scale=scale).pdf(gx), label=f'$\sigma = {scale}$', ax=g.ax)

With this in mind, we can now rethink about describing our data. For every input $\mathbf{x}_i$, we posit that the _likelihood_ of the output value being $y_i$ is given by,

$$
p(y_i \mid \mathbf{x}_i) = \mathcal{N}(y_i; f_\theta(\mathbf{x}_i), \sigma^2).
$$

In simpler words, we are saying that we are _uncertain_ about the output values (they are potentially noisy in the first place after all). Therefore, we would like to instead consider $f_\theta(\mathbf{x}_i)$ (the same as our previous function) as the most probable value, and in addition note the uncertainty around this mean value by a data-independent value fixed $\sigma^2$. A simplifying assumption we make is that all the outputs are _independently and identically_ distributed. This allows us to say that the total likelihood of all observed data is,

$$
p\left([y_1,y_2,\dots,y_n] \mid [\mathbf{x}_1,\mathbf{x}_2,\dots,\mathbf{x}_n] \right) = \prod_{i=1}^n p(y_i \mid \mathbf{x}_i)
$$

The _learning_ objective is therefore to find parameters $\theta$, such that we maximize the likelihood of the observed values $\mathbf{y}$. We instead maximize the $\log$-likelihood, which makes for a simpler maximization objective, and a surprising connection to the optimization problem we've seen before.

$$
\log{\prod_{i=1}^n p(y_i \mid \mathbf{x}_i)} = \sum_{i=1}^n \log{p(y_i \mid \mathbf{x}_i)} = -\frac{n}{2}\log{2\pi\sigma^2} - \frac{1}{2\sigma^2}\sum_{i=1}^n (y_i - f_\theta(\mathbf{x}_i))^2
$$

Ignoring the constant terms, we see that minimizing the _sum-of-squares_ loss and maximizing the $\log$-likelihood, is exactly the same objective! Further, we note that algebraicly, what we've done is consider a Gaussian noise corrupted version of $y = f_\theta(x)$, i.e.

$$
y = f(x) + \epsilon, \epsilon \sim \mathcal{N}(0, \sigma^2)
$$

Now, let's generate the data again, but this time with noise, and re-fit it using linear regression.

In [ ]:
m = 2.0
c = 5.0
sigma = 3.0

x = np.linspace(0.0, 10.0, 100)
y = m * x + c
y_obs = y + sigma * np.random.randn(*y.shape)

In [ ]:
g = sns.relplot(x=x, y=y_obs, label='Observations', color='gray')
sns.lineplot(x=x,y=y, ax=g.ax, label='True function')
g.ax.set_xlabel('x')
g.ax.set_ylabel('y');

In [ ]:
model = LinearRegression()
model.fit(np.expand_dims(x, axis=-1), y_obs)
model.coef_, model.intercept_

We see that we don't quite recover the exact values. This is because of the observation noise, and this is typical of the real world. In a noiseless world, we'd only require two observations to find the underlying parameter described by such a system. But in the real-world, the noisier the system, the more data it needs to arrive closest to the ground truth (and yet remain with some irreducible error after it all). We never really recover the true values, but hope that the values are good enough to serve the downstream purposes of our modeling.

We can visualize the predicted line. Increasing the noise $\sigma$ will make the distinction more apparent.

In [ ]:
y_pred = model.predict(np.expand_dims(x, axis=-1))
g = sns.relplot(x=x, y=y_obs, label='Observations', color='gray')
sns.lineplot(x=x,y=y, ax=g.ax, label='True function')
sns.lineplot(x=x,y=y_pred, ax=g.ax, label='Predicted function')
g.ax.set_xlabel('x')
g.ax.set_ylabel('y');

**NOTE**: Being exactly equal to the optimization approach, the probabilistic model may appear to be cumbersome and unnecessary. There is, however, more to the story but beyond the scope of this tutorial.